# Chapter 6 - The ARMA Model

## Listing 6-1. Getting the Births data into Python

In [ ]:
import pandas as pd

# Read the csv file
data = pd.read_csv('births_data.csv', sep=';')

# Keep useful columns
data = data[['Date', 'Births']]

## Listing 6-2. Aggregating the Births data to yearly data

In [ ]:
data['year'] = data.Date.apply(lambda x: x[-4:])
data = data[['Births', 'year']].groupby('year').sum()
data.head()


## Listing 6-3. Plotting the yearly Births data

In [ ]:
import matplotlib.pyplot as plt
ax = data.plot()
ax.set_ylabel('Births')
plt.show()


## Listing 6-4. Applying the ADF test to the Births yearly totals

In [ ]:
!pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data['Births'])
print(result)

pvalue = result[1]

if pvalue < 0.05:
    print('stationary')
else:
    print('not stationary')


## Listing 6-7. Creating the ACF and PACF plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

plot_acf(data['Births'], lags=40)

plot_pacf(data['Births'], lags=40)

plt.show()


## Listing 6-8. Fitting the ARMA(1,1) model

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA

# Forecast the first ARMA(1,1) model
mod = ARIMA(list(data['Births']), order=(1,0,1))
res = mod.fit()
pred = res.predict()

# Print the r2 score of the prediction
print(r2_score(data, pred))

# Create the plot
plt.plot(list(data['Births']))
plt.plot(pred)
plt.legend(['Actual Births', 'Predicted Births'])
plt.xlabel('Timesteps')
plt.show()


## Listing 6-9. Plotting a histrogram of the residuals

In [ ]:
ax = pd.Series(res.resid).hist()
ax.set_ylabel('Number of occurences')
ax.set_xlabel('Residual')
plt.show()


## Listing 6-10. Obtaining the summary table of your model’s fit

In [ ]:
res.summary()

## Listing 6-11. Grid search with cross-validation for optimal p and q

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit


def train_model(p, q):

    errors = []
    
    tscv = TimeSeriesSplit(test_size=10)
    
    for train_index, test_index in tscv.split(data_array):
        
        X_train, X_test = data_array[train_index], data_array[test_index]
        X_test_orig = X_test
        
        fcst = []
        for step in range(10):
            mod = ARIMA(X_train, order=(p,0,q))
            res = mod.fit()

            fcst.append(res.forecast(steps=1))

            X_train = np.concatenate((X_train, X_test[0:1,:]))
            X_test = X_test[1:]
            
        errors.append(r2_score(X_test_orig, fcst))
        
    pq_result = [p, q, np.mean(errors)]

    
    return pq_result



data_array = data.values

avg_errors = []

for p in range(13):
    for q in range(13):

        try:
            pq_result = train_model(p, q)
            print(pq_result)
            avg_errors.append(pq_result)

        except:
            print(p,q,'failure')

avg_errors = pd.DataFrame(avg_errors)
avg_errors.columns = ['p', 'q', 'error']
result = avg_errors.pivot(index='p', columns='q')['error']


In [ ]:
result

In [ ]:
# find max
maxi = 0
for p in range(12):
    for q in range(12):
        if result.iloc[p,q] > maxi:
            maxi = result.iloc[p,q]
            maxidx = (p,q)
print(maxi,maxidx)

## Listing 6-12. Showing the test prediction of the final model

In [ ]:
data_array = data.values
X_train, X_test = data_array[:-10], data_array[-10:]
X_test_orig = X_test

fcst = []
for step in range(10):
    mod = ARIMA(X_train, order=(2,0,9))
    res = mod.fit()
    fcst.append(res.forecast(steps=1))
    X_train = np.concatenate((X_train, X_test[0:1,:]))
    X_test = X_test[1:]

plt.plot(X_test_orig)
plt.plot(fcst)
plt.legend(['Actual Births', 'Predicted Births'])
plt.xlabel('Time steps of test data')
plt.show()


## Listing 6-13. Adding MLFlow

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

mlflow.autolog()

def train_model(p, q):
    
    errors = []
    tscv = TimeSeriesSplit(test_size=10)
    for train_index, test_index in tscv.split(data_array):
        X_train, X_test = data_array[train_index], data_array[test_index]
        X_test_orig = X_test
        fcst = []
        for step in range(10):
            mod = ARIMA(X_train, order=(p,0,q))
            res = mod.fit()
            fcst.append(res.forecast(steps=1))
            X_train = np.concatenate((X_train, X_test[0:1,:]))
            X_test = X_test[1:]
        errors.append(r2_score(X_test_orig, fcst))
    pq_result = [p, q, np.mean(errors)]
    return pq_result
    
with mlflow.start_run():
    
    data_array = data.values
    
    avg_errors = []
    
    for p in [2]:
        for q in [9]:
    
            try:
                pq_result = train_model(p, q)

                # Log this metric to mlflow
                mlflow.log_metric('cross-validation-r2-score', pq_result[2])
                avg_errors.append(pq_result)
    
            except:
                print(p,q,'failure')
    
    avg_errors = pd.DataFrame(avg_errors)
    avg_errors.columns = ['p', 'q', 'error']
    result = avg_errors.pivot(index='p', columns='q')['error']